In [ ]:
!pip install tensorflow


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

BASE_DIR = "/content/drive/MyDrive/Final_Project"
RAW_DIR = os.path.join(BASE_DIR, "data", "raw")
PROCESSED_DIR = os.path.join(BASE_DIR, "data", "processed")
MODELS_DIR = os.path.join(BASE_DIR, "models")
OUTPUT_JSON = os.path.join(BASE_DIR, "outputs", "json")
OUTPUT_PLOTS = os.path.join(BASE_DIR, "outputs", "plots")

# יצירת תיקיות אם לא קיימות
for path in [PROCESSED_DIR, MODELS_DIR, OUTPUT_JSON, OUTPUT_PLOTS]:
    os.makedirs(path, exist_ok=True)


### יצירת טבלת פיצ'רים מלאה (S&P + חדשות כלכליות)

In [ ]:
import pandas as pd
import numpy as np
import os

# נתיבים
BASE_DIR = "/content/drive/MyDrive/Final_Project"
RAW_DIR = os.path.join(BASE_DIR, "data", "processed")
PROCESSED_DIR = os.path.join(BASE_DIR, "data", "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)

snp_path = os.path.join(RAW_DIR, "SNP_DATA.csv")
news_path = os.path.join(RAW_DIR, "NEW_DATA_מוגמר.csv")
output_path = os.path.join(PROCESSED_DIR, "SNP_NEWS_FINAL.csv")

# --- שלב 1: קריאת נתונים ---
snp_df = pd.read_csv(snp_path, low_memory=False, on_bad_lines='skip')
news_df = pd.read_csv(news_path, low_memory=False)

# --- שלב 2: טיפוס תאריך ---
news_df['Date'] = pd.to_datetime(news_df['Date'], dayfirst=True, errors='coerce')
news_df = news_df[news_df['Date'].notna()]
snp_df['Date'] = pd.to_datetime(snp_df['Date'], dayfirst=True, errors='coerce')
snp_df = snp_df[snp_df['Date'].notna()].copy()

# --- שלב 3: יצירת פיצ'רים למדד ---
required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
for col in required_cols:
    snp_df[col] = pd.to_numeric(snp_df[col], errors='coerce')

snp_df['Pct_Change'] = snp_df['Close'].pct_change() * 100
snp_df['MA_5'] = snp_df['Close'].rolling(window=5).mean()
snp_df['STD_5'] = snp_df['Close'].rolling(window=5).std()

delta = snp_df['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
snp_df['RSI'] = 100 - (100 / (1 + rs))

ema_12 = snp_df['Close'].ewm(span=12, adjust=False).mean()
ema_26 = snp_df['Close'].ewm(span=26, adjust=False).mean()
snp_df['MACD'] = ema_12 - ema_26
snp_df['MACD_RSI_Ratio'] = snp_df['MACD'] / snp_df['RSI']
snp_df['Change_Pct'] = ((snp_df['Close'] - snp_df['Open']) / snp_df['Open']) * 100
snp_df['Volatility'] = snp_df[['Open', 'High', 'Low', 'Close']].std(axis=1)

# --- שלב 4: סנטימנט יומי ---
sentiment_map = {'bullish': 1, 'bearish': -1, 'neutral': 0}
news_df['overall_sentiment_label'] = news_df['overall_sentiment_label'].astype(str).str.lower().map(sentiment_map)
sentiment_col = [col for col in news_df.columns if 'overall_sentiment_score' in col][0]
news_df[sentiment_col] = pd.to_numeric(news_df[sentiment_col], errors='coerce')

daily_sentiment = news_df.groupby('Date')[sentiment_col].mean().reset_index()
daily_sentiment.rename(columns={sentiment_col: 'Sentiment_Score'}, inplace=True)
daily_label = news_df.groupby('Date')['overall_sentiment_label'].mean().reset_index()
daily_label.rename(columns={'overall_sentiment_label': 'sentiment_label'}, inplace=True)

# --- שלב 5: מיזוג ויצירת פיצ'רים נוספים ---
merged_df = pd.merge(snp_df, daily_sentiment, on="Date", how="left")
merged_df = pd.merge(merged_df, daily_label, on="Date", how="left")
merged_df['Is_News_Day'] = merged_df['Date'].isin(daily_sentiment['Date']).astype(int)

merged_df['Sentiment_Score'] = merged_df['Sentiment_Score'].fillna(method='ffill').fillna(0)
merged_df['sentiment_label'] = merged_df['sentiment_label'].fillna(method='ffill').fillna(0)

# --- שלב 6: דעיכה של Is_News_Day ---
decay_rate = 0.005
decay_values = []
last_value = 0
for _, row in merged_df.iterrows():
    if row['Is_News_Day'] == 1:
        last_value = 1
    else:
        last_value = max(0, last_value - decay_rate)
    decay_values.append(last_value)
merged_df['Is_News_Day'] = decay_values

# --- שלב 7: יצירת score_Day ---
merged_df['score_Day'] = merged_df['Is_News_Day'] * merged_df['Sentiment_Score']

# --- שלב 8: שמירה לקובץ ---
merged_df.to_csv(output_path, index=False)

# --- סטטוס ---
print("✅ טבלה סופית מוכנה עם כל הפיצ'רים:")
print("📊 שורות:", len(merged_df))
print("📅 טווח תאריכים:", merged_df['Date'].min().date(), "→", merged_df['Date'].max().date())
print("📁 נשמר אל:", output_path)


/tmp/ipython-input-83-662170529.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['overall_sentiment_label'] = news_df['overall_sentiment_label'].astype(str).str.lower().map(sentiment_map)
/tmp/ipython-input-83-662170529.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df[sentiment_col] = pd.to_numeric(news_df[sentiment_col], errors='coerce')
/tmp/ipython-input-83-662170529.py:65: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill()

✅ טבלה סופית מוכנה עם כל הפיצ'רים:
📊 שורות: 1574
📅 טווח תאריכים: 2018-12-10 → 2025-03-14
📁 נשמר אל: /content/drive/MyDrive/Final_Project/data/processed/SNP_NEWS_FINAL.csv


בדיקת תאריכים חוקיים של ימי מסחר

In [ ]:
import pandas as pd
import os

# נתיב לקובץ המעודכן
market_path = "/content/drive/MyDrive/Final_Project/data/processed/SNP_NEWS_FINAL.csv"

# קריאה
market_df = pd.read_csv(market_path)

# ניקוי מזהמים
market_df = market_df.loc[:, ~market_df.columns.str.contains('^Unnamed')]
market_df = market_df.drop(columns=[col for col in market_df.columns if col.strip() == ""])

# המרת עמודת Date
market_df['Date'] = pd.to_datetime(market_df['Date'], errors='coerce')
market_df = market_df[market_df['Date'].notna()].sort_values("Date")

# יצירת רשימת תאריכים חוקיים בפורמט אחיד YYYY-MM-DD
valid_market_dates = set(market_df["Date"].dt.strftime("%Y-%m-%d").tolist())

# תצוגה לדוגמה
print("✅ מספר ימי מסחר חוקיים:", len(valid_market_dates))
print("📅 תאריך ראשון:", min(valid_market_dates))
print("📅 תאריך אחרון:", max(valid_market_dates))


✅ מספר ימי מסחר חוקיים: 1574
📅 תאריך ראשון: 2018-12-10
📅 תאריך אחרון: 2025-03-14


הכנת הנתונים למודל + טעינה או אימון

In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
import tensorflow as tf
import random

# קביעת seed לשחזוריות
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# הגדרת נתיבים
BASE_DIR = "/content/drive/MyDrive/Final_Project"
DATA_PATH = os.path.join(BASE_DIR, "data", "processed", "SNP_NEWS_FINAL.csv")
MODELS_DIR = os.path.join(BASE_DIR, "models")
os.makedirs(MODELS_DIR, exist_ok=True)

SCALER_PATH = os.path.join(MODELS_DIR, "final_scaler.save")
MODEL_PATH = os.path.join(MODELS_DIR, "final_snp500_model.h5")

# טעינת הנתונים
df = pd.read_csv(DATA_PATH)
df = df.drop(columns=["Date"], errors='ignore')  # עמודת תאריך לא נדרשת למודל
df = df.apply(pd.to_numeric, errors='coerce').dropna()
features = df.columns.tolist()

# נרמול
scaler = MinMaxScaler()
scaler.fit(df)
scaled_data = scaler.transform(df)
joblib.dump(scaler, SCALER_PATH)

# הגדרת רצפים
sequence_length = 30
forecast_horizon = 7
X, y = [], []

for i in range(len(scaled_data) - sequence_length - forecast_horizon):
    X.append(scaled_data[i:i+sequence_length])
    y.append(scaled_data[i+sequence_length:i+sequence_length+forecast_horizon, features.index("Close")])

X, y = np.array(X), np.array(y)

# בניית מודל GRU
model = Sequential([
    GRU(128, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.3),
    GRU(64),
    Dropout(0.3),
    Dense(forecast_horizon)
])
model.compile(optimizer='adam', loss='mean_squared_error')

# אימון
model.fit(X, y, epochs=30, batch_size=32, verbose=1)

# שמירה
model.save(MODEL_PATH)
print("✅ המודל והסקיילר נשמרו בהצלחה.")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step - loss: 0.0910
Epoch 2/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - loss: 0.0175
Epoch 3/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - loss: 0.0123
Epoch 4/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 82ms/step - loss: 0.0108
Epoch 5/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - loss: 0.0087
Epoch 6/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - loss: 0.0084
Epoch 7/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 0.0068
Epoch 8/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 112ms/step - loss: 0.0058
Epoch 9/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - loss: 0.0052
Epoch 10/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss: 0.0051
Epoch 11/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 7s 107ms/step - loss: 0.0049
Epoch 12/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 8s 71ms/step - loss: 0.0047
Epoch 13/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 114ms/step - loss: 0.0045
Epoch 14/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 8s 71ms/step - loss: 0.0045
Epoch 15/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - loss: 0.0041


✅ המודל והסקיילר נשמרו בהצלחה.


חיזוי לתקופות שונות + שמירה כ־JSON

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from datetime import timedelta
import joblib
from tensorflow.keras.models import load_model

# === הגדרות נתיבים ===
BASE_DIR = "/content/drive/MyDrive/Final_Project"
DATA_PATH = os.path.join(BASE_DIR, "data", "processed", "SNP_NEWS_FINAL.csv")
SCALER_PATH = os.path.join(BASE_DIR, "models", "final_scaler.save")
MODEL_PATH = os.path.join(BASE_DIR, "models", "final_snp500_model.h5")
OUTPUT_DIR = os.path.join(BASE_DIR, "outputs")
JSON_DIR = os.path.join(OUTPUT_DIR, "json")
PLOT_DIR = os.path.join(OUTPUT_DIR, "plots")

# יצירת תיקיות במידת הצורך
os.makedirs(JSON_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)

# === קריאת נתונים ===
df = pd.read_csv(DATA_PATH)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.drop(columns=[col for col in df.columns if col.strip() == ""])
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df[df['Date'].notna()].sort_values("Date").reset_index(drop=True)

# רשימת ימי מסחר חוקיים
valid_market_dates = set(df["Date"].dt.strftime("%Y-%m-%d").tolist())
latest_real_date = df["Date"].max()

# === טעינת מודל וסקלר ===
scaler = joblib.load(SCALER_PATH)
model = load_model(MODEL_PATH)
features = [col for col in df.columns if col != 'Date']
scaled_data = scaler.transform(df[features])

sequence_length = 30
forecast_horizon = 7

# === פונקציית חיזוי ===
def forecast_sequence(start_idx, label, future=False):
    input_seq = scaled_data[start_idx:start_idx + sequence_length].reshape(1, sequence_length, len(features))
    predicted_scaled = model.predict(input_seq)[0]

    temp = np.zeros((forecast_horizon, len(features)))
    temp[:, features.index("Close")] = predicted_scaled
    forecast_unscaled = scaler.inverse_transform(temp)[:, features.index("Close")]

    # תאריכי תחזית
    if future:
        forecast_dates = [latest_real_date + timedelta(days=i) for i in range(1, forecast_horizon + 1)]
    else:
        anchor_date = df["Date"].iloc[start_idx + sequence_length - 1]
        forecast_dates = pd.date_range(end=anchor_date, periods=forecast_horizon)[::-1]

    # סינון תאריכים חוקיים (למעט עבור חיזוי עתידי)
    filtered_dates = [d for d in forecast_dates if d.strftime("%Y-%m-%d") in valid_market_dates] if not future else forecast_dates

    if len(filtered_dates) < forecast_horizon and not future:
        print(f"⚠️ אזהרה: רק {len(filtered_dates)} ימים חוקיים עבור {label}, אך שומרים את הקיים.")
    if len(filtered_dates) == 0:
        print(f"❌ אין תאריכים חוקיים בכלל עבור {label}, מדלגים.")
        return

    # יצירת DataFrame
    forecast_df = pd.DataFrame({
        'Date': filtered_dates,
        'Predicted_Close': forecast_unscaled[:len(filtered_dates)]
    })

    # חישוב אחוז שינוי
    start_price = forecast_df['Predicted_Close'].iloc[0]
    end_price = forecast_df['Predicted_Close'].iloc[-1]
    change_percent = ((end_price - start_price) / start_price) * 100
    recommendation = "BUY" if change_percent > 1 else "SELL" if change_percent < -1 else "HOLD"

    forecast_df["Date"] = forecast_df["Date"].dt.strftime("%Y-%m-%d")

    # === שמירת JSON ===
    output = {
        "data": forecast_df.to_dict(orient="records"),
        "recommendation": recommendation,
        "expectedChange": f"{change_percent:.2f}%"
    }
    json_path = os.path.join(JSON_DIR, f"{label}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(output, f, ensure_ascii=False, indent=2)

    # === גרף PNG ===
    plt.figure(figsize=(10, 4))
    plt.plot(forecast_df["Date"], forecast_df["Predicted_Close"], marker="o", color="blue")
    plt.title(f"S&P 500 Forecast – {label}")
    plt.xlabel("Date")
    plt.ylabel("Predicted Close Price")
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plot_path = os.path.join(PLOT_DIR, f"{label}.png")
    plt.savefig(plot_path)
    plt.close()

    print(f"✅ {label} – שמור ({recommendation} / {change_percent:.2f}%)")

# === הרצות חיזוי ===

# 1. תחזית עתידית (שבוע קדימה)
forecast_sequence(start_idx=len(df) - sequence_length, label="forecast_latest", future=True)

# 2. תחזיות היסטוריות
dates_back = {
    "forecast_week1": "2025-03-14",
    "forecast_week2": "2025-03-07",
    "forecast_week3": "2025-03-01"
}

for label, date_str in dates_back.items():
    target_date = pd.to_datetime(date_str)
    possible_dates = df[df["Date"] <= target_date]
    if not possible_dates.empty:
        end_idx = possible_dates.index[-1]
        if end_idx >= sequence_length:
            forecast_sequence(start_idx=end_idx - sequence_length, label=label)
        else:
            print(f"❌ לא מספיק נתונים עבור {label}")
    else:
        print(f"❌ לא נמצא תאריך חוקי עבור {label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
✅ forecast_latest – שמור (BUY / 1.03%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
⚠️ אזהרה: רק 5 ימים חוקיים עבור forecast_week1, אך שומרים את הקיים.
✅ forecast_week1 – שמור (HOLD / -0.25%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
⚠️ אזהרה: רק 5 ימים חוקיים עבור forecast_week2, אך שומרים את הקיים.
✅ forecast_week2 – שמור (HOLD / -0.28%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
⚠️ אזהרה: רק 5 ימים חוקיים עבור forecast_week3, אך שומרים את הקיים.
✅ forecast_week3 – שמור (HOLD / -0.18%)


סינון החיזויים לימי המסחר בפועל + יצור קובץ תחזיות מאוחד

In [ ]:
import json
import os
from datetime import datetime

# נתיב לקבצים החדשים בתוך outputs/json
json_dir = "/content/drive/MyDrive/Final_Project/outputs/json"

forecast_files = [
    os.path.join(json_dir, "forecast_latest.json"),
    os.path.join(json_dir, "forecast_week1.json"),
    os.path.join(json_dir, "forecast_week2.json"),
    os.path.join(json_dir, "forecast_week3.json")
]

# טוען את כולם
forecast_all = []

for file in forecast_files:
    if os.path.exists(file):
        with open(file, encoding='utf-8') as f:
            data = json.load(f)
            forecast_all.extend(data["data"])  # מתוך מפתח "data"
    else:
        print(f"⚠️ קובץ לא נמצא: {file}")

# הסרה של כפילויות ומיון לפי תאריך
forecast_all = {entry['Date']: entry for entry in forecast_all}  # מסיר כפולים לפי תאריך
forecast_all = list(forecast_all.values())
forecast_all.sort(key=lambda x: x['Date'])

# שמירה לקובץ חדש
output_path = os.path.join(json_dir, "forecast_all.json")
with open(output_path, "w", encoding='utf-8') as f:
    json.dump(forecast_all, f, indent=2, ensure_ascii=False)

print(f"✔️ נוצר קובץ forecast_all.json עם {len(forecast_all)} תאריכים.")


✔️ נוצר קובץ forecast_all.json עם 22 תאריכים.


שמירת נתוני שוק אמיתיים

In [ ]:
# שמירת מחירי הסגירה בפועל לחודש האחרון
start_date = pd.to_datetime("2025-02-14")
end_date = pd.to_datetime("2025-03-14")

actual_df = market_df.copy()
actual_df = actual_df[(actual_df['Date'] >= start_date) & (actual_df['Date'] <= end_date)]
actual_json = actual_df[['Date', 'Close']].copy()
actual_json['Date'] = actual_json['Date'].dt.strftime('%Y-%m-%d')

output_path = os.path.join(OUTPUT_JSON, "actual_data.json")
actual_json.to_json(output_path, orient="records", indent=2)

print(f"✅ actual_data.json נשמר ({len(actual_json)} תאריכים)")


✅ actual_data.json נשמר (20 תאריכים)


שמירת חדשות כלכליות

In [ ]:
import pandas as pd
import os
import json

# נתיב קובץ נתוני חדשות
RAW_NEWS_PATH = "/content/drive/MyDrive/Final_Project/data/raw/economic_news_cleaned.csv"
OUTPUT_JSON = "/content/drive/MyDrive/Final_Project/outputs/json"

# קריאה
news_df = pd.read_csv(RAW_NEWS_PATH)
news_df = news_df.loc[:, ~news_df.columns.str.contains('^Unnamed')]
news_df = news_df.drop(columns=[col for col in news_df.columns if col.strip() == ""])
news_df['Date'] = pd.to_datetime(news_df['Date'], errors='coerce', dayfirst=True)
news_df = news_df[news_df['Date'].notna()]

# עיבוד
news_json = news_df[['Date', 'Sentiment_Label', 'summary', 'source', 'title']].copy()
news_json['Date'] = news_json['Date'].dt.strftime('%Y-%m-%d')
news_json.columns = ['Date', 'Sentiment_Label', 'summary', 'Source', 'title']

# שמירה
os.makedirs(OUTPUT_JSON, exist_ok=True)
output_path = os.path.join(OUTPUT_JSON, "news_data.json")
news_json.to_json(output_path, orient="records", indent=2, force_ascii=False)

print(f"✅ news_data.json נשמר ({len(news_json)} ידיעות)")



✅ news_data.json נשמר (36625 ידיעות)


מייצר את market_dashboard_data.json

In [ ]:
import pandas as pd
import os
import json

# קלט ופלט
RAW = "/content/drive/MyDrive/Final_Project/data/processed/SNP_NEWS_FINAL.csv"
OUT = "/content/drive/MyDrive/Final_Project/outputs/json/market_dashboard_data.json"

# קריאה
df = pd.read_csv(RAW)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.drop(columns=[col for col in df.columns if col.strip() == ""])
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)
df = df[df['Date'].notna()].sort_values("Date").reset_index(drop=True)

# עיבוד
df = df[["Date", "Close", "Volume"]].copy()
df["MA20"] = df["Close"].rolling(window=20).mean()

# RSI
delta = df["Close"].diff()
gain = delta.where(delta > 0, 0).rolling(window=14).mean()
loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
rs = gain / loss
df["RSI"] = 100 - (100 / (1 + rs))

# MACD
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26

# המלצה
def reco(row):
    if row["RSI"] < 30 and row["MACD"] > 0:
        return "BUY"
    elif row["RSI"] > 70 and row["MACD"] < 0:
        return "SELL"
    else:
        return "HOLD"

df["Recommendation"] = df.apply(reco, axis=1)

# שמירה ל־JSON
result = []
for _, row in df.dropna().iterrows():
    result.append({
        "date": row["Date"].strftime("%Y-%m-%d"),
        "rsi": round(row["RSI"], 2),
        "macd": round(row["MACD"], 2),
        "volatility": int(row["Volume"]),
        "MA20": round(row["MA20"], 2),
        "Recommendation": row["Recommendation"]
    })

with open(OUT, "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

print("✅ market_dashboard_data.json נוצר ונשמר!")


✅ market_dashboard_data.json נוצר ונשמר!
